# 1. Dependency

In [ ]:
!pip install git+https://github.com/mayukh18/reco
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from reco.recommender import FunkSVD
from reco.metrics import rmse
import datetime
from collections import Counter
from datetime import timedelta

from google.colab import drive
drive.mount("/content/drive")

  Cloning https://github.com/mayukh18/reco to /tmp/pip-req-build-o4xdr61c
  Running command git clone -q https://github.com/mayukh18/reco /tmp/pip-req-build-o4xdr61c
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. Data preparation

In [ ]:
transaction = pd.read_csv("/content/drive/My Drive/Colab Notebooks/567_hw/data/transactions_train.csv",header=0, dtype={'article_id':str})

transaction["t_dat"] = pd.to_datetime(transaction["t_dat"])

customer_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/567_hw/data/customers_clustered.csv",header=0)
sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/567_hw/data/sample_submission.csv')
test_customer = ["0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d8cd0c725276a467a2a","ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e4747568cac33e8c541831","ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1778d0116cffd259264","ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab53481233731b5c4f8b7"]

## 2.1 Deal With Customers_Clusters

In [ ]:
customer_list = customer_df.groupby(["clusters"])["customer_id"].apply(list)

# 3 Rank articles with the Popularity

In [ ]:


def get_pop_items(transaction):
  # in recent 8 weeks
  train_pop = transaction.loc[transaction["t_dat"] >= datetime.datetime(2020, 7, 28)]
  # train_pop.loc[:, 'd'] = 0

  train_pop.loc[:,'num'] = (train_pop["t_dat"].max() - train_pop["t_dat"])
  train_pop['pop_factor'] = 1 / (train_pop['num'].dt.days + 1)
  popular_items_group = train_pop.groupby(["article_id"])['pop_factor'].sum()
  print(len(popular_items_group))
  _, popular_items = zip(*sorted(zip(popular_items_group, popular_items_group.keys()))[::-1])
  return popular_items

# popular_items = get_pop_items(transaction)

# 4. item2iem

In [ ]:
def get_most_freq_next_item(user_group):
    next_items = {}
    for user in tqdm(user_group.keys()):
        items = user_group[user]
        for i,item in enumerate(items[:-1]):
            if item not in next_items:
                next_items[item] = []
#             if item != items[i+1]:
#                 next_items[item].append(items[i+1])
            next_items[item].append(items[i+1])
    
    pred_next = {}
    for item in next_items:
        if len(next_items[item]) >= 5:
            most_common = Counter(next_items[item]).most_common()
            ratio = most_common[0][1]/len(next_items[item])
            if ratio >= 0.1:
                pred_next[item] = most_common[0][0]
            
    return pred_next

In [ ]:
# recent 4 weeks;
one_day = timedelta(days=1)
year_month_day = str(transaction["t_dat"].max() - one_day*7*4)[:10].split('-')

year_ = int(year_month_day[0])
month_ = int(year_month_day[1])
day_ = int(year_month_day[2])
def get_user_group(transaction):
  user_group = transaction.loc[transaction["t_dat"] >= datetime.datetime(year_, month_, day_)].groupby(["customer_id"])["article_id"].apply(list)
  pred_next = get_most_freq_next_item(user_group)
  user_group_dict = user_group.to_dict()
  return pred_next, user_group_dict

# pred_next, user_group_dict = get_user_group(transaction)

## 4.1 FunkSVD

In [ ]:
def get_df(transaction):
# recent 16 weeks;
  df = transaction.loc[transaction["t_dat"] >= datetime.datetime(2020, 2, 2), ["customer_id", "article_id", "t_dat"]].copy()

  #time decay popularity of each article
  df['num'] = (df["t_dat"].max() - df["t_dat"])
  df['pop_factor'] = 1 / (df['num'].dt.days + 1)

  # df = df[df['customer_id'].isin(test_customer)]

  popular_items_group = df.groupby(["article_id"])['pop_factor'].sum()

  # purchase count of each article
  items_total_count = df.groupby(["article_id"])["article_id"].count()

  # purchase count of each user
  users_total_count = df.groupby(["customer_id"])["customer_id"].count()
  users_total_count

  df['pop_score'] = 1
  df = df.groupby(["customer_id", "article_id"]).sum().reset_index()
  df['pop_score'] = df.apply(lambda row: row['pop_score']/popular_items_group[row["article_id"]], axis=1)


  df['pop_score'] = df['pop_score'].apply(lambda x: 5.0 if x>5.0 else x)
  df = df[["customer_id", "article_id", 'pop_score']]
  df = df.sample(frac=1).reset_index(drop=True)
  return df
# df = get_df(transaction)

In [ ]:
# shuffling
def get_svd(df):
  svd = FunkSVD(k=8, learning_rate=0.008, regularizer = .01, iterations = 80, method = 'stochastic', bias=True)
  svd.fit(X=df, formatizer={'user':"customer_id", 'item':"article_id", 'value':'pop_score'},verbose=True)
  return svd
# svd = get_svd(df)

## 4.2 Recently Purchase most

In [ ]:
def get_spilit_transactions(transaction):
  train1 = transaction.loc[(transaction["t_dat"] >= datetime.datetime(2020, 9, 15))]
  train2 = transaction.loc[(transaction["t_dat"] >= datetime.datetime(2020, 9, 1)) & (transaction["t_dat"] < datetime.datetime(2020, 9, 15))]
  train3 = transaction.loc[(transaction["t_dat"] >= datetime.datetime(2020, 6, 1)) & (transaction["t_dat"] < datetime.datetime(2020, 9, 1))]
  train4 = transaction.loc[(transaction["t_dat"] >= datetime.datetime(2020, 2, 25)) & (transaction["t_dat"] < datetime.datetime(2020, 6, 1))]

  tmp = train1.groupby(["customer_id","article_id"])["t_dat"].agg('count').reset_index()
  tmp.columns = ["customer_id","article_id",'cnt']
  train1 = train1.merge(tmp, on = ["customer_id","article_id"], how='left')
  train1 = train1.sort_values(["t_dat", 'cnt'],ascending=False)
  train1.index = range(len(train1))
  positive_items_per_user1 = train1.groupby(["customer_id"])["article_id"].apply(list)


  tmp = train2.groupby(["customer_id","article_id"])["t_dat"].agg('count').reset_index()
  tmp.columns = ["customer_id","article_id",'cnt']
  train2 = train2.merge(tmp, on = ["customer_id","article_id"], how='left')
  train2 = train2.sort_values(["t_dat", 'cnt'],ascending=False)
  train2.index = range(len(train2))
  positive_items_per_user2 = train2.groupby(["customer_id"])["article_id"].apply(list)


  tmp = train3.groupby(["customer_id","article_id"])["t_dat"].agg('count').reset_index()
  tmp.columns = ["customer_id","article_id",'cnt']
  train3 = train3.merge(tmp, on = ["customer_id","article_id"], how='left')
  train3 = train3.sort_values(["t_dat", 'cnt'],ascending=False)
  train3.index = range(len(train3))
  positive_items_per_user3 = train3.groupby(["customer_id"])["article_id"].apply(list)


  tmp = train4.groupby(["customer_id","article_id"])["t_dat"].agg('count').reset_index()
  tmp.columns = ["customer_id","article_id",'cnt']
  train4 = train4.merge(tmp, on = ["customer_id","article_id"], how='left')
  train4 = train4.sort_values(["t_dat", 'cnt'],ascending=False)
  train4.index = range(len(train4))
  positive_items_per_user4 = train4.groupby(["customer_id"])["article_id"].apply(list)

  return positive_items_per_user1, positive_items_per_user2, positive_items_per_user3, positive_items_per_user4

# positive_items_per_user1, positive_items_per_user2, positive_items_per_user3, positive_items_per_user4 = get_spilit_transactions(transaction)

In [ ]:
sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/567_hw/data/sample_submission.csv')

In [ ]:

def get_recommend(svd, positive_items_per_user1, positive_items_per_user2, positive_items_per_user3, positive_items_per_user4,sub,popular_items, user_group_dict, pred_next):
  result = []

  userindexes = {svd.users[i]:i for i in range(len(svd.users))}
  for user in tqdm(sub["customer_id"].unique()):

      user_output = []
      if user in positive_items_per_user1.keys():
          most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user1[user]).most_common()}
          user_index = userindexes[user]
          new_order = {}
          for k in list(most_common_items_of_user.keys())[:20]:
              try:
                  itemindex = svd.items.index(k)
                  pred_value = np.dot(svd.userfeatures[user_index], svd.itemfeatures[itemindex].T) + svd.item_bias[0, itemindex]
              except:
                  pred_value = most_common_items_of_user[k]
              new_order[k] = pred_value
          user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]
          
      elif user in positive_items_per_user2.keys():
          most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user2[user]).most_common()}
          user_index = userindexes[user]
          new_order = {}
          for k in list(most_common_items_of_user.keys())[:20]:
              try:
                  itemindex = svd.items.index(k)
                  pred_value = np.dot(svd.userfeatures[user_index], svd.itemfeatures[itemindex].T) + svd.item_bias[0, itemindex]
              except:
                  pred_value = most_common_items_of_user[k]
              new_order[k] = pred_value
          user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]
          
      elif user in positive_items_per_user3.keys():
          most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user3[user]).most_common()}
          user_index = userindexes[user]
          new_order = {}
          for k in list(most_common_items_of_user.keys())[:20]:
              try:
                  itemindex = svd.items.index(k)
                  pred_value = np.dot(svd.userfeatures[user_index], svd.itemfeatures[itemindex].T) + svd.item_bias[0, itemindex]
              except:
                  pred_value = most_common_items_of_user[k]
              new_order[k] = pred_value
          user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]
          
      elif user in positive_items_per_user4.keys():
          most_common_items_of_user = {k:v for k, v in Counter(positive_items_per_user4[user]).most_common()}
          user_index = userindexes[user]
          new_order = {}
          for k in list(most_common_items_of_user.keys())[:20]:
              try:
                  itemindex = svd.items.index(k)
                  pred_value = np.dot(svd.userfeatures[user_index], svd.itemfeatures[itemindex].T) + svd.item_bias[0, itemindex]
              except:
                  pred_value = most_common_items_of_user[k]
              new_order[k] = pred_value
          user_output += [k for k, v in sorted(new_order.items(), key=lambda item: item[1])][:12]
      
      if user in user_group_dict:
          item_his = user_group_dict[user][::-1]
          for item in item_his:
              if item in pred_next and pred_next[item] not in user_output:
                  user_output += [pred_next[item]]
      if len(user_output) > 12:
          user_output = user_output[:12]
          
      if len(user_output) < 12:
          user_output += list(popular_items[:12 - len(user_output)])
      
      assert(len(user_output) == 12) 
      user_output = ' '.join(user_output)
      result.append([user, user_output])
  return result

# result = get_recommend(svd, positive_items_per_user1, positive_items_per_user2, positive_items_per_user3, positive_items_per_user4, sub, popular_items, user_group_dict, pred_next)

In [ ]:
# result = pd.DataFrame(result)
# result.columns = ["customer_id", 'prediction']
# result

# 5.1 Deal with different clusters

In [ ]:
def get_transactions(customer_list, transaction):
  transactions = []
  subs = []
  for customer in customer_list:
    temp = transaction[transaction['customer_id'].isin(customer)]
    sub_temp = sub[sub['customer_id'].isin(customer)]
    transactions.append(temp)
    subs.append(sub_temp)
  return transactions, subs

transactions, subs = get_transactions(customer_list, transaction)
results = []
def deal_with_cluster(transactions, subs):
  for index, transaction in enumerate(transactions):
    sub = subs[index]
    popular_items = get_pop_items(transaction)
    pred_next, user_group_dict = get_user_group(transaction)
    df = get_df(transaction)
    svd = get_svd(df)
    positive_items_per_user1, positive_items_per_user2, positive_items_per_user3, positive_items_per_user4 = get_spilit_transactions(transaction)
    result = get_recommend(svd, positive_items_per_user1, positive_items_per_user2, positive_items_per_user3, positive_items_per_user4, sub, popular_items, user_group_dict, pred_next)
    results.append(result)

deal_with_cluster(transactions, subs)


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


28837


100%|██████████| 80106/80106 [00:00<00:00, 136077.21it/s]


Epoch 0: Error: 0.5327787485675759
Epoch 1: Error: 0.3781551606495175
Epoch 2: Error: 0.3353166132098909
Epoch 3: Error: 0.31051881026900247
Epoch 4: Error: 0.2914264701014261
Epoch 5: Error: 0.2748744878182246
Epoch 6: Error: 0.2598721599158777
Epoch 7: Error: 0.24613530371410985
Epoch 8: Error: 0.2336211688551037
Epoch 9: Error: 0.2223184278507695
Epoch 10: Error: 0.21217735376644012
Epoch 11: Error: 0.20311138545734914
Epoch 12: Error: 0.19502060339585686
Epoch 13: Error: 0.18780162958858648
Epoch 14: Error: 0.1813462232806437
Epoch 15: Error: 0.17555760501004186
Epoch 16: Error: 0.17034615304075587
Epoch 17: Error: 0.1656418491750966
Epoch 18: Error: 0.1613833619149681
Epoch 19: Error: 0.15751139854587662
Epoch 20: Error: 0.15397929562635318
Epoch 21: Error: 0.1507477977063061
Epoch 22: Error: 0.14778243556004245
Epoch 23: Error: 0.14505112569896572
Epoch 24: Error: 0.1425295872817051
Epoch 25: Error: 0.14019392369815842
Epoch 26: Error: 0.1380243533550924
Epoch 27: Error: 0.136005

100%|██████████| 457319/457319 [28:32<00:00, 267.02it/s]


28685


100%|██████████| 80111/80111 [00:00<00:00, 138428.69it/s]


Epoch 0: Error: 0.5350351180958257
Epoch 1: Error: 0.3796891839311912
Epoch 2: Error: 0.3366006289944505
Epoch 3: Error: 0.3115192089067954
Epoch 4: Error: 0.29212556671195994
Epoch 5: Error: 0.27529745221039664
Epoch 6: Error: 0.26005923679402915
Epoch 7: Error: 0.24613690947732375
Epoch 8: Error: 0.23347427927965242
Epoch 9: Error: 0.22205380706272218
Epoch 10: Error: 0.21182606868434192
Epoch 11: Error: 0.20269793864674968
Epoch 12: Error: 0.19455870445629364
Epoch 13: Error: 0.18729503263386982
Epoch 14: Error: 0.18079912426171624
Epoch 15: Error: 0.1749742383662565
Epoch 16: Error: 0.16973437850102086
Epoch 17: Error: 0.16500307149042806
Epoch 18: Error: 0.16071724118621877
Epoch 19: Error: 0.15682139387422286
Epoch 20: Error: 0.1532681630559143
Epoch 21: Error: 0.1500161692728314
Epoch 22: Error: 0.14703175337000735
Epoch 23: Error: 0.14428456277518845
Epoch 24: Error: 0.1417490051012912
Epoch 25: Error: 0.13940060706714716
Epoch 26: Error: 0.1372205731896213
Epoch 27: Error: 0.1

 49%|████▉     | 225990/457334 [13:08<11:22, 338.94it/s]

In [ ]:


result = pd.DataFrame(results[0])
result.columns = ["customer_id", 'prediction']
result1 = pd.DataFrame(results[1])
result1.columns = ["customer_id", 'prediction']
result2 = pd.DataFrame(results[2])
result2.columns = ["customer_id", 'prediction']
result1 = result1.append(result)
result1 = result1.append(result2)
result1.reset_index(inplace=True, drop=True)
result1

## 5.2 Result Submission

In [ ]:
result1.to_csv("/content/drive/My Drive/Colab Notebooks/567_hw/output/submission_test.csv", index=False)